In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit

from threading import Thread, Lock

/Users/danilo/.conda/envs/bcb/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


## Dataset Configuration

In [2]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

C = d_clinical.values

y_d = outputs["dement_fail"].values
y_c = outputs["cvd_fail"].values

In [3]:
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y_d))

C_ = C[tr_idx]
C_test = C[ts_idx]

y_d_ = y_d[tr_idx]
y_d_test = y_d[ts_idx]

y_c_ = y_c[tr_idx]
y_c_test = y_c[ts_idx]

In [4]:
ds_list = [C_]
ds_test = [C_test]
ds_names = ['clinic']

## Kernel Definition

In [5]:
kernel_names = ['linear', 'laplacian']
kernel_type = [{'linear':[1], 'laplacian':[0.2, 0.4]},
               {'linear':[1], 'laplacian':[0.3, 0.6]}]

## Other shared parameters initialization

In [6]:
estimator = ca.centeredKernelAlignment
lock_dementia = Lock()
lock_cardio = Lock()
valid_fold = 3
threads = []
pen_params = [0.7, 1]

## Thread

In [7]:
def child(sampler,estimator,ds_list,ds_test,y_d_,y_d_test,y_c_,y_c_test,lock_dementia,lock_cardio,valid_fold,exclusion_list,verbose,approach):
    """
    #DEMENTIA
    
    result1 = sampler.sample(kernel_type, estimator, ds_list, y_d_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names)
    result1.performancesFeatures(fileToWrite = 'results_temp/Dementia_train.txt', header = '\nDementia Laplacian - Gaussian\n' + approach + '\n', lock = lock_dementia)
    ut.testConfigurations(estimator, y_d_, y_d_test, w_list, ds_list, ds_test, kernel_names, lamb_list, sparsity, 'classification', lock_dementia, fileToWrite = 'results_temp/Dementia_test.txt', header = 'Dementia Linear, Laplacian, Sigmoid, Polynomial, Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    """   
    # CARDIO
    
    result1 = sampler.sample(kernel_type, estimator, ds_list, y_c_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names)
    result1.performancesFeatures(fileToWrite = 'results_temp/Cardio_train.txt', header = '\nCardio Laplacian - Gaussian\n' + approach + '\n', lock = lock_cardio)
    ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names, lamb_list, sparsity, 'classification', lock_cardio, fileToWrite = 'results_temp/Cardio_test.txt', header = 'Cardio Linear, Laplacian, Sigmoid, Polynomial, Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)


## L2, Centering, Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - Normalizing'))
t.start()
threads.append(t)

## L2, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Normalizing - K Normalizing'))
t.start()
threads.append(t)

## L2, Centering, K-Normalization

In [8]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [9]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - K Normalizing'))
t.start()
threads.append(t)

## L2, Centering, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - Normalizing - K Normalizing'))
t.start()
threads.append(t)

In [10]:
for t in threads:
    t.join()

print("Operations completed")
threads = []

winner:  [1, 1]
winning dict:  [{'clinic': {'linear': 1, 'laplacian': 0.2}}, {'clinic': {'linear': 1, 'laplacian': 0.3}}]
winning param list:  [[[1, 0.2]], [[1, 0.3]]]
winning pen params:  [1, 1]
{'config': {'linear': [[1], [1], [1]], 'laplacian': [[0.2], [0.2], [0.4]]}, 'CA': (0.025031155961147022, 8.075089493301079e-07), 'Accuracy': (0.5600718527585112, 6.927379717036723e-05), 'Precision': (0.38387028555089797, 3.54868915706564e-05), 'Recall': (0.622093023255814, 0.00047322877230935803), 'eta': (array([-0.04405075,  0.99901993]), array([1.86750346e-05, 3.39804683e-08])), 'lambda': 1}
{'config': {'linear': [[1], [1], [1]], 'laplacian': [[0.3], [0.3], [0.6]]}, 'CA': (0.02489928988528682, 5.959950286517154e-07), 'Accuracy': (0.5576465388276893, 7.904505194077637e-05), 'Precision': (0.3819089804491264, 3.967257356160804e-05), 'Recall': (0.6182170542635659, 0.0005934138573403042), 'eta': (array([-0.02354956,  0.99948961]), array([4.65820773e-04, 1.24464092e-07])), 'lambda': 1}
Operations 

## L1, Centering, Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - Normalizing'))
t.start()
threads.append(t)

## L1, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Normalizing - K Normalizing'))
t.start()
threads.append(t)

## L1, Centering, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - K Normalizing'))
t.start()
threads.append(t)

## L1, Centering, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - Normalizing - K Normalizing'))
t.start()
threads.append(t)

In [ ]:
for t in threads:
    t.join()

print("Operations completed")
threads = []